In [1]:
import tensorflow as tf
import numpy as np
import config as cfg
import csv
import pandas as pd
from PIL import Image
import cv2
import os

In [2]:
def read_whole_csv(path):
    file = open(path)
    csvreader = csv.reader(file)
    all = []
    next(csvreader, None)
    for row in csvreader:
        ye = True
        for value in row:
            if value.startswith('-'):
                ye = False
                break
        if ye:
            all.append([row[0], [int(row[1]), int(row[2])],
                       int(row[3]) - int(row[1]), row[5:]])
    file.close()
    return all


In [3]:
def split_dataset(dataset, test_ratio=0.20):
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]


In [4]:
def crop_face(image, x, y, w, h):
    image = image[y:y + h, x:x + w]
    return image


In [71]:
def prep_image(dp):
    wh = int(dp[2])
    points = [(float(a) / wh) for a in dp[3]]
    # crop out face from image
    image = cv2.imread(cfg.IMAGES_PATH + dp[0])
    image = crop_face(image, int(dp[1][0]), int(dp[1][1]), wh, wh)

    # resize image to xxx * xxx
    image = Image.fromarray(image)
    image = image.resize((cfg.CROP_SIZE, cfg.CROP_SIZE))
    image = np.array(image)

    # (should I really convert it to tensor?)
    points = tf.convert_to_tensor(points, dtype=tf.float32)

    # PROFIT?
    return image, points


In [5]:
# def prep_image_old(dp):
#     image_path = dp[0]
#     bb = dp[1]
#     wh = dp[2]
#     points = dp[3]
#     # crop out face from image
#     image = cv2.imread(cfg.IMAGES_PATH + image_path)
#     image = crop_face(image, int(
#         bb[0]), int(bb[1]), int(wh), int(wh))
#     path = 'data\data\\tmp\\' + image_path
#     cv2.imwrite(path, image)

#     # resize image to xxx * xxx
#     image = Image.open(path)
#     image = image.resize((194, 194))

#     path = 'data\data\\croped\\' + image_path
#     image.save(path)
#     # save resize ratio
#     crop_r = cfg.CROP_SIZE / wh

#     # resize points using resize ratio
#     for point in points:
#         point = float(point)
#         point = point * crop_r

#     # convert points to relative [0.0 -> 1]
#     npo = []
#     for point in points:
#         point_ = float(point) / wh
#         npo.append(point_)

#     # (should I really convert it to tensor?)
#     npo = tf.convert_to_tensor(npo, dtype=tf.float32)

#     # PROFIT?
#     return path, npo


In [6]:
ds = read_whole_csv(cfg.LABELS_PATH)
ds = pd.DataFrame(ds)
train_ds, test_ds = split_dataset(ds)

In [20]:
# # old
# trainX = []
# trainY = []
# testX = []
# testY = []

# for i in range(train_ds.shape[0]):
#     path, points = prep_image_old(train_ds.iloc[i])
#     input_image = tf.io.read_file(path)
#     input_image = tf.image.decode_image(input_image)
#     input_image = tf.cast(input_image, dtype=tf.int32)
#     trainX.append(input_image)
#     trainY.append(points)

#     # VERY TEMPOPARY BREAK
#     # break

# for i in range(test_ds.shape[0]):
#     path, points = prep_image_old(train_ds.iloc[i])
#     input_image = tf.io.read_file(path)
#     input_image = tf.image.decode_image(input_image)
#     input_image = tf.cast(input_image, dtype=tf.int32)
#     testX.append(input_image)
#     testY.append(points)

In [21]:
# trainX_ = tf.convert_to_tensor(trainX, dtype=tf.float32) / 255
# trainY_ = tf.convert_to_tensor(trainY, dtype=tf.float32)

# testX_ = tf.convert_to_tensor(testX, dtype=tf.float32) / 255
# testY_ = tf.convert_to_tensor(testY, dtype=tf.float32)


In [72]:
# new
trainX = []
trainY = []
testX = []
testY = []

for i in range(train_ds.shape[0]):
    image, points = prep_image(train_ds.iloc[i])
    input_image = tf.cast(image, dtype=tf.int32)
    trainX.append(input_image)
    trainY.append(points)

for i in range(test_ds.shape[0]):
    image, points = prep_image(train_ds.iloc[i])
    input_image = tf.cast(image, dtype=tf.int32)
    testX.append(input_image)
    testY.append(points)

trainX_ = tf.convert_to_tensor(trainX, dtype=tf.float32) / 255
trainY_ = tf.convert_to_tensor(trainY, dtype=tf.float32)

testX_ = tf.convert_to_tensor(testX, dtype=tf.float32) / 255
testY_ = tf.convert_to_tensor(testY, dtype=tf.float32)


KeyboardInterrupt: 

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense


def build(size):
    model = Sequential()
    inputShape = (size, size, 3)
    # CONV => RELU => POOL
    model.add(Conv2D(32, (3, 3), padding="same",
                     input_shape=inputShape))
    model.add(Activation("relu"))
    # model.add(MaxPooling2D(pool_size=(3, 3)))
    # model.add(Dropout(0.25))

    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))

    # model.add(MaxPooling2D(pool_size=(2, 2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))

    # (CONV => RELU) * 2 => POOL
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    # model.add(Conv2D(128, (3, 3), padding="same"))
    # model.add(Activation("relu"))
    # model.add(Dropout(0.25))
    
    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    # model.add(Dropout(0.5))
    model.add(Dense(cfg.OUTPUTS))
    return model


In [23]:
def get_model(load_latest = False):
    model = build(194)
    model.compile(optimizer='adam',
                loss=tf.keras.losses.Huber(),
                metrics=['accuracy'])

    if load_latest:
        checkpoint_dir = os.path.dirname(cfg.CHECKPOINT_PATH)
        latest = tf.train.latest_checkpoint(checkpoint_dir)
        model.load_weights(latest)
       
    batch_size = 77
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=cfg.CHECKPOINT_PATH,
        verbose=1,
        save_weights_only=True,
        save_freq=5*batch_size)
    return model, cp_callback


def load_model(path):
    model = tf.keras.models.load_model(path)
    return model

def evaluate_model(model, x, y):
    loss, acc = model.evaluate(testX_, testY_, verbose=2)
    print(f'Model, accuracy: {acc*100:5.2f}%')

def save_model(model, path):
    model.save(path)


In [ ]:
model, cp_callback = get_model()

model.fit(trainX_, 
          trainY_,
          epochs=10,
          validation_data=(testX_, testY_),
          callbacks=[cp_callback])



In [14]:
def predictor(model, image_path, size):
    image = cv2.imread(image_path)
    # dims = image.shape
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(30, 30)
    )
    dimsCrop = []
    for (x, y, w, h) in faces:
        image = image[y:y + h, x:x + w]
        dimsCrop.append([x,y,w,h])
    
    image = Image.fromarray(image)
    image = image.resize((size, size))
    image = np.array(image)
    
    # predict
    input_image = tf.cast(image, dtype=tf.float32) / 255
    input_image = tf.expand_dims(input_image, axis=0)
    res = model.predict(input_image)

    x_ = []
    y_ = []
    for i in range(0, 136, 2):
        x_.append(int(res[0][i] * size * (dimsCrop[0][3] / size) + dimsCrop[0][0]))
        y_.append(int(res[0][i+1]* size * (dimsCrop[0][3] / size) + dimsCrop[0][1]))

    image = cv2.imread(image_path)
    for i in range(68):
        image = cv2.circle(image, (x_[i], y_[i]), 1, (0, 0, 255), int(image.shape[1] * 0.006))
    cv2.imwrite('pic.jpeg', image)
    return res
    

# resnew = predictor(model, 'data\data\InitialDS\\1 (10).jpg', 194)
resold = predictor(nm, 'data\data\InitialDS\\0 (19).jpg', 194)
# print(resold)

[[[110 112 113]
  [112 114 115]
  [110 112 113]
  ...
  [ 33  37  41]
  [ 29  33  39]
  [ 23  27  32]]

 [[110 112 113]
  [111 113 114]
  [111 112 113]
  ...
  [ 24  29  32]
  [ 20  24  29]
  [ 20  24  29]]

 [[110 112 113]
  [111 112 114]
  [110 112 114]
  ...
  [ 22  26  32]
  [ 27  31  37]
  [ 24  27  33]]

 ...

 [[115 117 118]
  [115 117 118]
  [115 117 118]
  ...
  [ 69  90 121]
  [ 78 102 132]
  [ 84 108 138]]

 [[115 117 118]
  [115 117 118]
  [115 117 118]
  ...
  [ 75  96 127]
  [ 82 106 136]
  [ 89 113 143]]

 [[116 118 119]
  [115 117 118]
  [115 117 118]
  ...
  [ 82 103 134]
  [ 87 110 141]
  [ 93 117 147]]]


In [4]:
nm = load_model('best_model_so_far194.h5')